In [ ]:
#
# Copyright (C) 2024 by Sonja Filiposka <sonja.filiposka@finki.ukim.mk>
#
# This code is licensed under a Creative Commons Attribution 4.0 International License. (see LICENSE.txt for details)
#
# General Description - this notebook is used to extract several attributes information from OMNET output vector file
# It creates an output file that contains values of a set of attributes, named "distance", "measuredSinrDl", "measuredSinrUl", 
#   "rcvdSinrDl", "averageCqiDl", "servingCell", "rlcDelayDl", "rlcPacketLossTotal", "rlcPduDelayDl", "rlcPduPacketLossDl",
#   "rlcPduThroughputDl", "rlcThroughputDl", "receivedPacketFromLowerLayer" from OMNeT++ output vectors
#

In [1]:
# Import necessary libraries
import pandas as pd

In [2]:
# OMNET data
# corresponding file to SUMO
# Time    Object  Vector  Value
# 0.1     car[0].cellularNic.phy  "servingCell """""      0.0
# 0.1     car[0].cellularNic.nrChannelModel[0]    "distance """"" 730.321606
# 0.102   car[0].cellularNic.nrChannelModel[0]    "measuredSinrUl """""   31.197934
# 2.142   car[0].cellularNic.nrChannelModel[0]    "measuredSinrDl """""   20.288022
# 0.104   car[0].cellularNic.nrPhy        "averageCqiDl """""     10.0
# 0.104   car[0].cellularNic.nrChannelModel[0]    "rcvdSinrDl """""       23.440494
# 0.104   car[0].cellularNic.nrMac        "receivedPacketFromLowerLayer """""     73.0
# 0.107   car[0].cellularNic.nrRlc.um     "rlcPduThroughputDl """""       0.0
# 0.107   car[0].cellularNic.nrRlc.um     "rlcPduDelayDl """""    0.004
# 0.107   car[0].cellularNic.nrRlc.um     "rlcThroughputDl """""  644.859813
# 0.107   car[0].cellularNic.nrRlc.um     "rlcPacketLossTotal """""       0.0
# 0.107   car[0].cellularNic.nrRlc.um     "rlcDelayDl """""       0.007
# 0.107   car[0].cellularNic.nrRlc.um     "rlcPduPacketLossDl """""       0.0
# 0.107   car[0].cellularNic.pdcpRrc      "receivedPacketFromLowerLayer """""     69.0
# 

In [3]:
# remove ].cellularNic.*
!sed -i -e "s/].cellularNic.[^ ]*\t/\t/" 4928_omnet_AI.csv

In [4]:
# remove receivedPacketFromLowerLayer lines
!sed -i -e "/receivedPacketFromLowerLayer/d" 4928_omnet_AI.csv

In [5]:
# remove car[
!sed -i -e "s/car\[//g" 4928_omnet_AI.csv

In [6]:
# remove double quotes from OMNET file before read
!cat 4928_omnet_AI.csv |  tr -d \" >> 4928_omnet_AI_noquotes.csv

In [ ]:
# Load data from csv file (SUMO with classification and 999999)
file_path = '4928_omnet_AI_noquotes.csv'
omnet = pd.read_csv(file_path, delimiter='\t')

# Display the first few rows of the data
print(omnet.head())

In [8]:
transposedomnet = omnet.groupby(['Time','Object','Vector'])['Value'].aggregate('min').unstack()

In [9]:
print(omnet.head())

   Time  Object        Vector        Value
0   0.1       0  servingCell      0.000000
1   0.1       0     distance    615.016313
2   0.1       0     distance    655.315612
3   0.1       0     distance   1136.230193
4   0.1       0     distance    177.124825


In [10]:
transposedomnet = transposedomnet.reset_index()
print(transposedomnet.head())

Vector   Time  Object  averageCqiDl    distance   measuredSinrDl   \
0       0.100       0            NaN  177.124825              NaN   
1       0.102       0            NaN         NaN         26.68636   
2       0.104       0           13.0  177.124825              NaN   
3       0.107       0            NaN         NaN              NaN   
4       0.108       0            NaN         NaN         26.68636   

Vector  measuredSinrUl   rcvdSinrDl   rlcDelayDl   rlcPacketLossTotal   \
0                   NaN          NaN          NaN                  NaN   
1             38.116766          NaN          NaN                  NaN   
2                   NaN    30.359326          NaN                  NaN   
3                   NaN          NaN        0.007                  0.0   
4             38.116766          NaN          NaN                  NaN   

Vector  rlcPduDelayDl   rlcPduPacketLossDl   rlcPduThroughputDl   \
0                  NaN                  NaN                  NaN   
1   

In [11]:
transposedomnet = transposedomnet.set_index(['Object','Time'])
print(transposedomnet.head())

Vector        averageCqiDl    distance   measuredSinrDl   measuredSinrUl   \
Object Time                                                                 
0      0.100            NaN  177.124825              NaN              NaN   
       0.102            NaN         NaN         26.68636        38.116766   
       0.104           13.0  177.124825              NaN              NaN   
       0.107            NaN         NaN              NaN              NaN   
       0.108            NaN         NaN         26.68636        38.116766   

Vector        rcvdSinrDl   rlcDelayDl   rlcPacketLossTotal   rlcPduDelayDl   \
Object Time                                                                   
0      0.100          NaN          NaN                  NaN             NaN   
       0.102          NaN          NaN                  NaN             NaN   
       0.104    30.359326          NaN                  NaN             NaN   
       0.107          NaN        0.007                  0.0      

In [ ]:
transposedomnetF = transposedomnet.groupby("Object").fillna(method="ffill")
transposedomnetF = transposedomnetF.groupby("Object").fillna(method="bfill")
print(transposedomnetF)

In [ ]:
transposedomnetF = transposedomnetF.reset_index()
print(transposedomnetF.head())

In [ ]:
transposedomnetF.isnull().values.any()

In [ ]:
#sort by time
transposedomnetF = transposedomnetF.sort_values(by=['Time', 'Object'])

In [ ]:
# shift column 'Name' to first position 
first_column = transposedomnetF.pop('Time') 
  
# insert column using insert(position,column_name, 
# first_column) function 
transposedomnetF.insert(0, 'Time', first_column) 

In [ ]:
test = transposedomnetF

In [ ]:
test['Time'] = pd.to_timedelta(test['Time'], unit='s')

In [ ]:
test = test.set_index('Time')

In [ ]:
test=pd.DataFrame(test)

In [ ]:
list(test.columns)

In [ ]:
#drop all zeros
test = test.drop(columns=['rlcPacketLossTotal ','rlcPduPacketLossDl ','rlcPduThroughputDl '])

In [ ]:
final = test.groupby('Object', as_index=False).resample("10ms").mean()

In [ ]:
print(final.head())

In [ ]:
final.isnull().values.any()

In [ ]:
final = final.dropna()

In [ ]:
final.isnull().values.any()

In [ ]:
final = final.reset_index()
final['Time']=final['Time'].dt.total_seconds()

In [ ]:
print(final.head())

In [ ]:
final = final.drop(columns=['level_0'])

In [ ]:
print(final.head())

In [ ]:
final.rename(columns={'averageCqiDl ':'averageCqiDl','distance ':'distance','measuredSinrDl ':'measuredSinrDl','measuredSinrUl ':'measuredSinrUl','rcvdSinrDl ':'rcvdSinrDl','rlcDelayDl ':'rlcDelayDl','rlcPduDelayDl ':'rlcPduDelayDl','rlcThroughputDl ':'rlcThroughputDl','servingCell ':'servingCell'}, inplace=True)
list(final.columns)

In [ ]:
# add previous positions (-servingCell1 to -servingCell7)
# -1 means that the car is not connect to servingCell
final['servingCell-1'] = final.groupby('Object')['servingCell'].shift(100).fillna(-1)
final['servingCell-2'] = final.groupby('Object')['servingCell'].shift(200).fillna(-1)
final['servingCell-3'] = final.groupby('Object')['servingCell'].shift(300).fillna(-1)
final['servingCell-4'] = final.groupby('Object')['servingCell'].shift(400).fillna(-1)
final['servingCell-5'] = final.groupby('Object')['servingCell'].shift(500).fillna(-1)
final['servingCell-6'] = final.groupby('Object')['servingCell'].shift(600).fillna(-1)
final['servingCell-7'] = final.groupby('Object')['servingCell'].shift(700).fillna(-1)
final.head()

In [ ]:
# step by step change 999999 with last seen coordinates
final['servingCell1'] = final.groupby('Object')['servingCell'].shift(-100).fillna(-1)
final['servingCell2'] = final.groupby('Object')['servingCell'].shift(-200).fillna(-1)
final['servingCell3'] = final.groupby('Object')['servingCell'].shift(-300).fillna(-1)
final['servingCell4'] = final.groupby('Object')['servingCell'].shift(-400).fillna(-1)
final['servingCell5'] = final.groupby('Object')['servingCell'].shift(-500).fillna(-1)
final['servingCell6'] = final.groupby('Object')['servingCell'].shift(-600).fillna(-1)
final['servingCell7'] = final.groupby('Object')['servingCell'].shift(-700).fillna(-1)
final.head()

In [ ]:
# save to new file
output = '4928_omnet_AI_matrix.csv'
final.to_csv(output, index=False, sep='\t')

In [ ]:
#to check the file
# Load data from Excel file
file_path = '4928_omnet_AI_matrix.csv'
data = pd.read_csv(file_path, delimiter='\t', nrows=200)
# save to new file
output = '4928_omnet_AI_small.csv'
data.to_csv(output, index=False, sep='\t')